## Other Algorisms: LOCI, NMF, ICA

Another algorism, named Localized Optimized Combination of Images (LOCI, see Lafreniere et al. 2007) has been frequently used in high-contrast imaging.

In our course, we have also talked about the Nonnegative Matrix Factorization (NMF) and the Independent Component Analysis (ICA). How do they work on the high-contrast imaging?

The number of papers found about each method about PSF modelling:
![klip](KLIP.png)
![loci](LOCI.png)
![pca](PCA.png)
![ica](ICA.png)
![nmf](NMF.png)

### Localized Optimized Combination of Images (LOCI)

Males (2017) showed that LOCI is equivalent to PCA where all components are used.

In LOCI, the image is divided into pieces of areas. This can also be done when applying PCA.

(The figure is adopted from Lafreniere et al. 2007)
![lociarea](LOCI_subsection.png)

### A Quick Look at the Output of ICA and NMF

We will need SWarp (https://www.astromatic.net/software/swarp) in the data reduction. The installation is a bit annoying... You can skip the next cell if you want (the results are shown later as png images).

In [1]:
### Read the data ###

import os
import numpy as np
from scipy.optimize import minimize
from astropy.io import fits
import matplotlib.pyplot as plt
import glob
import pyklip.instruments.GPI as GPI

from sklearn.decomposition import NMF
from sklearn.decomposition import FastICA
from sklearn.decomposition import RandomizedPCA

cwd=os.getcwd()
#data_dir=cwd+'/../HD10647_J_131117'
data_dir=cwd+'/betaPic_J_131210'

filelist = glob.glob(data_dir+"/*.fits.gz")
dataset = GPI.GPIData(filelist, highpass=True)

Reading File: /home/ymh/astr502/finalproject/Ast-502-final-project/betaPic_J_131210/S20131210S0106_spdc_distorcorr.fits.gz
Reading File: /home/ymh/astr502/finalproject/Ast-502-final-project/betaPic_J_131210/S20131210S0105_spdc_distorcorr.fits.gz
Reading File: /home/ymh/astr502/finalproject/Ast-502-final-project/betaPic_J_131210/S20131210S0097_spdc_distorcorr.fits.gz
Reading File: /home/ymh/astr502/finalproject/Ast-502-final-project/betaPic_J_131210/S20131210S0094_spdc_distorcorr.fits.gz
Reading File: /home/ymh/astr502/finalproject/Ast-502-final-project/betaPic_J_131210/S20131210S0099_spdc_distorcorr.fits.gz
Reading File: /home/ymh/astr502/finalproject/Ast-502-final-project/betaPic_J_131210/S20131210S0107_spdc_distorcorr.fits.gz
Reading File: /home/ymh/astr502/finalproject/Ast-502-final-project/betaPic_J_131210/S20131210S0098_spdc_distorcorr.fits.gz
Reading File: /home/ymh/astr502/finalproject/Ast-502-final-project/betaPic_J_131210/S20131210S0111_spdc_distorcorr.fits.gz
Reading File: /h

In [2]:
### Preparing the data ###

imagelist=[]
imagecube=dataset.input[1::37]
size=imagecube[0].size
for index in range(len(imagecube)):
    imagelist.append(imagecube[index].reshape(size))
imagelist=np.array(imagelist).T
imagelist[np.isnan(imagelist)]=0
imagelist[np.isinf(imagelist)]=0
imagelist_nmf=imagelist.copy()
imagelist_nmf[imagelist_nmf<0]=0


In [21]:
### Run the algorisms ###

ica = FastICA(n_components=10)
pca=RandomizedPCA(n_components=10)
nmf=NMF(n_components=10)

ica_model=ica.fit(imagelist)
pca_model=pca.fit(imagelist)
nmf_model=nmf.fit(imagelist_nmf)

ica_result=ica.transform(imagelist)
pca_result=pca.transform(imagelist)
nmf_result=nmf.transform(imagelist_nmf)

/home/ymh/anaconda2/lib/python2.7/site-packages/sklearn/utils/deprecation.py:52: DeprecationWarning: Class RandomizedPCA is deprecated; RandomizedPCA was deprecated in 0.18 and will be removed in 0.20. Use PCA(svd_solver='randomized') instead. The new implementation DOES NOT store whiten ``components_``. Apply transform to get them.
  warnings.warn(msg, category=DeprecationWarning)


In [22]:
def nmf_residual(coeff,component,image):
    return np.sum((np.dot(coeff,component)-image)**2)


def show_results(datacube=imagecube, method='pca', n_components=10):

    filenamelist=[]

    #print dataset.filenames[::37]
    fitmodel_dict={'pca':pca_model,\
                     'ica':ica_model,\
                     'nmf':nmf_model}
    
    fitdata_dict={'pca':pca_result,\
                     'ica':ica_result,\
                     'nmf':nmf_result}

    fitdata=fitdata_dict[method]
    fitmodel=fitmodel_dict[method]
    for raw_index in range(len(imagecube)):
        i=0
        raw_image=imagecube[raw_index]
        raw_image[np.isnan(raw_image)]=0
        if method in ['pca','ica']:
            raw_image=raw_image-np.mean(raw_image)

        raw_row=raw_image.reshape(raw_image.size)
        
        fitdata_truncated=fitdata.T[:n_components]
        #'''
        if method=='nmf':
            ### nmf requires non-negative coefficients, so the code here is a bit different ###
            fitdata_coeff_init=np.linalg.lstsq(np.transpose(fitdata_truncated),np.transpose(raw_row))[0]
            res=minimize(nmf_residual,x0=fitdata_coeff_init,args=(fitdata_truncated,raw_row),\
                                  method='SLSQP',bounds=[(0,None)]*n_components)
            #print res.success
            fitdata_coeff=res.x
            fitdata_coeff=np.transpose(fitdata_coeff)
        else:
            fitdata_coeff=np.linalg.lstsq(np.transpose(fitdata_truncated),np.transpose(raw_row))[0]
        fitdata_reconstruct=np.transpose(np.dot(np.transpose(fitdata_truncated),fitdata_coeff))
        fitdata_reconstruct=fitdata_reconstruct.reshape(raw_image.shape)
        
        #'''
        #coeffs=fitdata[raw_index]
        #fitdata_reconstruct=fitmodel.inverse_transform(coeffs)[0]
        #print fitdata_reconstruct.shape
        #fitdata_reconstruct=fitdata_reconstruct.reshape(raw_image.shape)
        
        fitsdata=fits.open(dataset.filenames[raw_index*37+1])
        #print raw_index,dataset.filenames[raw_index*37+1]
        #fitsdata=fits.PrimaryHDU(data=raw_image)
        fitsdata[1].data=raw_image-fitdata_reconstruct
        if os.path.exists(data_dir+'/save/tmp'+str(raw_index)+'.fits'):
            os.system('rm '+data_dir+'/save/tmp'+str(raw_index)+'.fits')
        fitsdata.writeto(data_dir+'/save/tmp'+str(raw_index)+'.fits')
        filenamelist.append(data_dir+'/save/tmp'+str(raw_index)+'.fits')
        
    np.savetxt('swarp_input',filenamelist,fmt='%s')
    os.system('swarp @swarp_input -IMAGEOUT_NAME ./coadd'+method+str(n_components)+'.fits')
    
    return 'coadd'+method+str(n_components)+'.fits'
        

In [24]:
show_results(method='ica',n_components=10)
show_results(method='pca',n_components=10)
show_results(method='nmf',n_components=10)

'coaddnmf10.fits'

#### NMF
![nmf](NMF_sub.png)

#### ICA
![ica](ICA_sub.png)

#### PCA
![pca](PCA_sub.png)

### Independent Component Analysis (ICA)

Recall that ICA decompose the signal to several statistically irrelavant components (the crocktail party problem).

How can we intepret this process in PSF modelling?
... I do not know. Probably this is the reason why ICA does not work well.


### Nonnegative Matrix Factorization (NMF)

NMF tries to decompose the signal into several non-negative parts. It is useful when intepreting a signal that is a combination of some non-negative parts.

It is not difficult to intepret NMF in PSF modeling, since PSF can be regarded as a combination of several patterns (for examples, rings of an Airy pattern).

However, when we are trying to MODEL the shape of a PSF, rather than EXPLAIN why the PSF looks like this, NMF is less straightforward than PCA.

## Summary

Anyway, it seems that naively applying ICA or NMF is not a good choice for high-contrast imaging... At least not as good as PCA.